## Importing Tools

In [2]:
import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt

## EDA

* ### Data Properties

In [ ]:
# Shapes of train and test data (columns x rows)
print('Train shape:', train.shape)
print('Test shape:', test.shape)

# Describe the target variable (data statistics)
print(train.describe())

# Train distribution of passengers within rides
# counts the numbber of appereances of each value
print(train.passenger_count.value_counts())

In [ ]:
# Find the median price by the interest level
prices = train.groupby('interest_level', as_index = False)['price'].median()

* ### Features 

Utilizar [coeficientes](https://operdata.com.br/blog/coeficientes-de-correlacao/) para descobrir a relação entre as features e o target e plotar o gráfico.

In [3]:
for col in colunas:
    print( ' %16s   %12.4f   %9.4f' % (col, 
                                       pearsonr(dados[col],dados['preco'])[0], 
                                       pearsonr(dados[col],dados['preco'])[1]))

NameError: name 'colunas' is not defined

## Scoring

In [ ]:
# For each of 3 trained models
for model in [xg_depth_2, xg_depth_8, xg_depth_15]:
    # Make predictions
    train_pred = model.predict(dtrain)     
    test_pred = model.predict(dtest)          
    
    # Calculate metrics
    mse_train = mean_squared_error(train['sales'], train_pred)                  
    mse_test = mean_squared_error(test['sales'], test_pred)
    print('MSE Train: {:.3f}. MSE Test: {:.3f}'.format(mse_train, mse_test))

In [ ]:
# Import StratifiedKFold
from sklearn.model_selection import StratifiedKFold

# Create a StratifiedKFold object
str_kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=123)

# Loop through each split
fold = 0
for train_index, test_index in str_kf.split(train, train['interest_level']):
    # Obtain training and testing folds
    cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
    print('Fold: {}'.format(fold))
    print('CV train shape: {}'.format(cv_train.shape))
    print('Medium interest listings in CV train: {}\n'.format(sum(cv_train.interest_level == 'medium')))
    fold += 1

* ### Validation Pipeline

In [ ]:
fold_metrics = []

for train_index, test_index in str_kf.split(train):
    cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
    # Train Model
    model.fit(cv_train)
    # Make predictions
    predictions = model.predict(cv_test)
    # Calculate the metric
    metric = evaluate(cv_test, predictions)
    fold_metrics.append(metric)
    
# Overall Validation Score
overall_score_minimizing = np.mean(fold_metrics) + np.std(fold_metrics)